<a href="https://colab.research.google.com/github/bellaroseee/starry-bird/blob/master/gpuServerProxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `nst.py`

In [ ]:
import os
import tensorflow as tf

import numpy as np
import PIL.Image
import time


def get_styled_picture(content, style, regularize=False):
    """Return combined picture given the content and style image
    Args:
        content (str): path to content image (PIL.Image)
        style (str): path to style image (PIL.Image)
        regularize (bool, optional): whether or not we will regularize the picture. Defaults to False
    Returns:
        image (PIL.Image): combined image of style and content image
    """

    @tf.function()
    def train_step(image):
        with tf.GradientTape() as tape:
            outputs = extractor(image)
            loss = style_content_loss(outputs, style_targets, content_targets, style_weight,
                                      content_weight, num_style_layers, num_content_layers)
            if (regularize):
                loss += total_variation_weight * tf.image.total_variation(image)

        grad = tape.gradient(loss, image)
        opt.apply_gradients([(grad, image)])
        image.assign(clip_0_1(image))

    # Load the content and style image
    content_image = load_img(content)
    style_image = load_img(style)

    # define the layers to extract features from
    content_layers = ['block5_conv3']

    style_layers = ['block1_conv2',
                    'block2_conv2',
                    'block3_conv2',
                    'block4_conv2',
                    'block5_conv2']

    num_style_layers = len(style_layers)
    num_content_layers = len(content_layers)

    # extract content and style
    extractor = StyleContentModel(style_layers, content_layers)
    style_targets = extractor(style_image)['style']
    content_targets = extractor(content_image)['content']
    image = tf.Variable(content_image)

    # set optimizers and weights
    opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
    style_weight = 1e-2
    content_weight = 1e4
    # for regularization: 
    total_variation_weight = 30

    # keep track of the time for analysis purposes
    start = time.time()

    epochs = 10
    steps_per_epoch = 100

    for n in range(epochs):
        for m in range(steps_per_epoch):
            train_step(image)
        print(f"Training steps done: {n * steps_per_epoch}")

    end = time.time()
    print("Total time: {:.1f}".format(end - start))

    return tensor_to_image(image)


def get_multiple_styled_picture(content, styles, regularize=False):
    """
    Performs NST on multiple image inputs
    Args:
        content (str): path to content image (PIL.Image)
        styles (list(str)): paths to style images (PIL.Image)
        regularize (bool, optional): whether or not we will regularize the picture. Defaults to False
    Returns:
        image (PIL.Image): combined image of style and content image
    """

    # load the content image
    content_image = load_img(content)

    content_layers = ['block5_conv3']

    style_layers = ['block1_conv2',
                    'block2_conv2',
                    'block3_conv2',
                    'block4_conv2',
                    'block5_conv2']

    for style in styles:
        @tf.function()
        def train_step(image):
            with tf.GradientTape() as tape:
                outputs = extractor(image)
                loss = style_content_loss(outputs, style_targets, content_targets, style_weight, 
                            content_weight, num_style_layers, num_content_layers)
                if (regularize):
                    loss += total_variation_weight*tf.image.total_variation(image)

            grad = tape.gradient(loss, image)
            opt.apply_gradients([(grad, image)])
            image.assign(clip_0_1(image))

        style_image = load_img(style)

        num_style_layers = len(style_layers)
        num_content_layers = len(content_layers)

        # extract content and style
        style_targets = None
        content_targets = None
        extractor = StyleContentModel(style_layers, content_layers)
        style_targets = extractor(style_image)['style']
        content_targets = extractor(content_image)['content']
        image = tf.Variable(content_image)

        # set optimizers and weight
        opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
        style_weight = 1e-2
        content_weight = 1e4
        # for regularization: 
        total_variation_weight = 30

        # keep track of the time for analysis purposes
        start = time.time()

        epochs = 10 if len(styles) < 10 else len(styles)
        steps_per_epoch = 100

        for n in range(epochs // len(styles)):
            for m in range(steps_per_epoch):
                train_step(image)
            print(f"Training steps done: {n * steps_per_epoch}")

        end = time.time()
        print("Total time: {:.1f}".format(end - start))

        content_image = tf.convert_to_tensor(image)

    return tensor_to_image(image)


def tensor_to_image(tensor):
    """Returns an image from a tensor
    Args:
        tensor (tensorflow.python.framework.ops.EagerTensor): tensor to turn into image
    Returns:
        img (PIL.Image): resultant image
    """
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)


def load_img(img_pil):
    """Returns a tensor of the image
    Args:
        img_pil (string): path to image to turn to tensor
    Returns:
        img (tensorflow.python.framework.ops.EagerTensor): tensor
    """
    max_dim = 512
    img = tf.io.read_file(img_pil)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img


def vgg_layers(layer_names):
    """Creates a vgg model that returns a list of intermediate output values.
    Args:
        layer_names (list(str)): layer names from the VGG model
    Returns:
        model (`obj`): tensorflow keras model of the VGG with the new inputs.
    """
    # Load our model. Load pretrained VGG, trained on imagenet data
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    # don't want to further train the vgg
    vgg.trainable = False

    outputs = [vgg.get_layer(name).output for name in layer_names]

    model = tf.keras.Model([vgg.input], outputs)
    return model


def gram_matrix(input_tensor):
    """Calculates the gram matrix from the input tensor.
    gram matrix represents the style of an image calculated from feature maps
    """
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1] * input_shape[2], tf.float32)
    return result / (num_locations)


class StyleContentModel(tf.keras.models.Model):
    """Creates a model based on vgg 19.
    """
    def __init__(self, style_layers, content_layers):
        super(StyleContentModel, self).__init__()
        self.vgg = vgg_layers(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg.trainable = False

    def call(self, inputs):
        "Expects float input in [0,1]"
        inputs = inputs * 255.0
        preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed_input)
        style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                          outputs[self.num_style_layers:])

        style_outputs = [gram_matrix(style_output)
                         for style_output in style_outputs]

        content_dict = {content_name: value
                        for content_name, value
                        in zip(self.content_layers, content_outputs)}

        style_dict = {style_name: value
                      for style_name, value
                      in zip(self.style_layers, style_outputs)}

        return {'content': content_dict, 'style': style_dict}


def clip_0_1(image):
    """Clamps image values to between 0.0 to 1.0
    Args:
        image (PIL.Image): Preprocessed image
    Returns:
        new image (PIL.Image): Clamped image
    """
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)


def style_content_loss(outputs, style_targets, content_targets, style_weight,
                       content_weight, num_style_layers, num_content_layers):
    """Calculates the loss with weighed sum of mean square error from outputs 
    relative to targets 
    """
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name] - style_targets[name]) ** 2)
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name] - content_targets[name]) ** 2)
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss


# `server.py`

In [ ]:
import base64
import io

from flask import Flask, request, jsonify
from flask_cors import CORS
from flask_ngrok import run_with_ngrok
from nst import *

# Initialize the Flask application
app = Flask(__name__)
# apply CORS filter
CORS(app)
run_with_ngrok(app)

# route http posts to this method
@app.route('/api/upload', methods=['POST'])
def upload_file():
    # get the uploaded file from the client (instance of Werkzeug FileStorage). It should be under the 'file' tag
    uploaded_file = request.files['file']

    # it's the hackiest way possible, basically saves the picture and then we'll reflect it back to the client
    if uploaded_file.filename != '':
        uploaded_file.save(os.path.join('tmp', uploaded_file.filename))

    return "Files Received!", 200


@app.route('/api/nst', methods=['POST'])
def perform_nst():
    data = request.json
    base_image = data['base_image']
    style_images = data['style_images']

    path_to_base_image = os.path.join('tmp', base_image)
    path_to_style_images = []

    if not os.path.exists(path_to_base_image):
        return "Invalid Request", 400

    for img_name in style_images:
        path_res = os.path.join('tmp', img_name)
        if not os.path.exists(path_res):
            return "Invalid Request", 400
        path_to_style_images.append(path_res)

    img = None

    if len(path_to_style_images) == 1:
        img = get_styled_picture(content=path_to_base_image, style=path_to_style_images[0])
    else:
        img = get_multiple_styled_picture(content=path_to_base_image, styles=path_to_style_images)

    # clean up all temporary files
    for f in os.listdir('tmp'):
        os.remove(os.path.join('tmp', f))

    # img = Image.open(os.path.join('tmp', uploaded_file.filename))
    raw_bytes = io.BytesIO()
    img.save(raw_bytes, "JPEG")
    raw_bytes.seek(0)
    img_base64 = base64.b64encode(raw_bytes.read())

    return jsonify({'image': str(img_base64)}), 200


def create_tmp_if_not_exists():
    if not os.path.exists('tmp'):
        os.makedirs('tmp')
        return True
    return False


# start flask app
if __name__ == "__main__":
    if create_tmp_if_not_exists():
        print("Created tmp folder...")
    print("Starting server")
    app.run()
